In [67]:
import os
import sys
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI
import re 
from langchain_community.graphs import Neo4jGraph
import dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import numpy as np 
import pandas as pd
# from project import web_crawl
import subprocess
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# connection to API's 

In [68]:

#LLM API's 
dotenv.load_dotenv()
llm=AzureChatOpenAI(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    temperature=0,
    
)

graph=Neo4jGraph(url=os.getenv('NEO4J_URI_ONLINE'),
                 username=os.getenv('NEO4J_USERNAME_ONLINE'),
                 password=os.getenv('NEO4J_PASSWORD_ONLINE')
                #  database=os.getenv('NEO4J_DATABASE')
)


In [69]:
# empty graph database
# DElete everything from the database 
# delete_cypher=""" 
# Match(n)
# DETACH DELETE n
# """
# graph.query(delete_cypher)

# extract information from weburl


In [70]:
from langchain_core.documents import Document
from helpers.clean_data import clean_text
# url_input=("https://en.wikipedia.org/wiki/Elon_Musk")
# url_input=('https://en.wikipedia.org/wiki/Mark_Zuckerberg')
# url_input=('https://en.wikipedia.org/wiki/Bill_Gates')
# url_input=('https://en.wikipedia.org/wiki/Jeff_Bezos')
# url_input=('https://en.wikipedia.org/wiki/Steve_Jobs')
# url_input=('https://en.wikipedia.org/wiki/Sam_Altman')
# url_input=('https://en.wikipedia.org/wiki/Mukesh_Ambani')
# url_input=('https://en.wikipedia.org/wiki/Jensen_Huang')
# url_input=('https://en.wikipedia.org/wiki/Satoshi_Nakamoto')
# url_input=('https://en.wikipedia.org/wiki/Donald_Trump')
# url_input=('https://en.wikipedia.org/wiki/Larry_Ellison')
url_input=('https://en.wikipedia.org/wiki/Dana_White')
loader=WebBaseLoader([url_input])
data=loader.load().pop().page_content
data
    

'\n\n\n\nDana White - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life and education\n\n\n\n\n\n\n\n\n2\nCareer\n\n\n\n\nToggle Career subsection\n\n\n\n\n\n2.1\nUltimate Fighting Championship\n\n\n\n\n\n

In [71]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

data=clean_text(data)
documents=[Document(page_content=data)]
# print(documents)
splitter=RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=100)
smaller_doc=splitter.split_documents(documents)
len(smaller_doc)

11

# setting up prompt template example

In [72]:
template="""
you are an intelliegnt bot who can analyze any text with person information {doc} , 
your job is to read and analyse the information and create a json dictionary.
dictionary has the followinmg key:
Name:
Company:
Skills:
output must be json nothing else
"""

In [73]:
prompt=PromptTemplate(template=template,input_variables=['doc'])
llmchain=LLMChain(llm=llm,prompt=prompt)

In [74]:
for doc in smaller_doc[:2]:
    response=llmchain.invoke({'doc':doc.page_content})
    print(response['text'])
    

{
  "Name": "Dana Frederick White Jr",
  "Company": "Ultimate Fighting Championship",
  "Skills": [
    "Business Management",
    "Mixed Martial Arts",
    "Promotion",
    "Leadership",
    "Entrepreneurship"
  ]
}
{
  "Name": "Dana White",
  "Company": "Ultimate Fighting Championship",
  "Skills": [
    "Boxing",
    "Jiujitsu",
    "Management",
    "Promotion"
  ]
}


# from the predefined prompts

In [75]:
from helpers.prompts import graphPrompt
results=[]
for doc in smaller_doc[:30]:
    results.append(graphPrompt(doc.page_content))
results

{
    "entities": [
        {
            "label": "Dana White",
            "id": "Dana White",
            "role": "CEO and President of UFC",
            "description": "American businessman and CEO and president of the Ultimate Fighting Championship (UFC)"
        },
        {
            "label": "Ultimate Fighting Championship",
            "id": "Ultimate Fighting Championship",
            "role": "Organization",
            "description": "Global mixed martial arts organization"
        },
        {
            "label": "Power Slap",
            "id": "Power Slap",
            "role": "Organization",
            "description": "Slap fighting promotion owned by Dana White"
        },
        {
            "label": "Anne Stella",
            "id": "Anne Stella",
            "role": "Spouse",
            "description": "Dana White's wife since 1996"
        },
        {
            "label": "Bishop Gorman High School",
            "id": "Bishop Gorman High School",
            "r

[{'entities': [{'label': 'Dana White',
    'id': 'Dana White',
    'role': 'CEO and President of UFC',
    'description': 'American businessman and CEO and president of the Ultimate Fighting Championship (UFC)'},
   {'label': 'Ultimate Fighting Championship',
    'id': 'Ultimate Fighting Championship',
    'role': 'Organization',
    'description': 'Global mixed martial arts organization'},
   {'label': 'Power Slap',
    'id': 'Power Slap',
    'role': 'Organization',
    'description': 'Slap fighting promotion owned by Dana White'},
   {'label': 'Anne Stella',
    'id': 'Anne Stella',
    'role': 'Spouse',
    'description': "Dana White's wife since 1996"},
   {'label': 'Bishop Gorman High School',
    'id': 'Bishop Gorman High School',
    'role': 'Education',
    'description': 'High school attended by Dana White'},
   {'label': 'Lorenzo Fertitta',
    'id': 'Lorenzo Fertitta',
    'role': 'Acquaintance',
    'description': 'Met Dana White at Bishop Gorman High School'},
   {'label'

In [76]:
for res in results:
    print(res)


{'entities': [{'label': 'Dana White', 'id': 'Dana White', 'role': 'CEO and President of UFC', 'description': 'American businessman and CEO and president of the Ultimate Fighting Championship (UFC)'}, {'label': 'Ultimate Fighting Championship', 'id': 'Ultimate Fighting Championship', 'role': 'Organization', 'description': 'Global mixed martial arts organization'}, {'label': 'Power Slap', 'id': 'Power Slap', 'role': 'Organization', 'description': 'Slap fighting promotion owned by Dana White'}, {'label': 'Anne Stella', 'id': 'Anne Stella', 'role': 'Spouse', 'description': "Dana White's wife since 1996"}, {'label': 'Bishop Gorman High School', 'id': 'Bishop Gorman High School', 'role': 'Education', 'description': 'High school attended by Dana White'}, {'label': 'Lorenzo Fertitta', 'id': 'Lorenzo Fertitta', 'role': 'Acquaintance', 'description': 'Met Dana White at Bishop Gorman High School'}, {'label': 'June White', 'id': 'June White', 'role': 'Family', 'description': "Dana White's mother, 

In [77]:
# empty neo
#graph.query(delete_cypher)

# inject data into neo4j

In [78]:
from neo4j import GraphDatabase


In [79]:
driver=GraphDatabase.driver(uri=os.getenv('NEO4J_URI_ONLINE'),auth=(os.getenv('NEO4J_USERNAME_ONLINE'),os.getenv('NEO4J_PASSWORD_ONLINE')))
existing_set=[]
def sanitize_label(label):
    sanitized=re.sub(r"[^\w]", "_", label)
    return re.sub(r"_+", "_", sanitized).strip("_")

def insert_entities(session,entities):
    for entity in entities:
        
        sanitize_labels=sanitize_label(entity['id'])
        query=f"""
            MERGE (n:{sanitize_labels} {{id:$id}})
            ON CREATE SET
                n.role=$role,
                n.description=$description
            ON MATCH SET
                n.role=$role,
                n.description=$description

            """
        session.run(query,label=entity['label'], id=entity['id'],role=entity['role'],description=entity['label'])
        print(f'Inserted entity: {entity["id"]}')
        existing_set.append(entity['label'])
    
def insert_relationship(session,relationships):
    for relation in relationships:
        query = f"""
        MATCH (a {{id: $source}}), (b {{id: $target}})
        MERGE (a)-[:{sanitize_label(relation['type'])} {{
            id: $id,
            description: $description
        }}]->(b)
        """
        session.run(query,source=relation['source'],target=relation['target'],id=relation['id'],description=relation['description'])
        print(f'Inserted relationship: {relation["id"]}')


In [80]:
with driver.session() as session:
    for result in results:
        try:
            session.execute_write(insert_entities,result['entities'])
            session.execute_write(insert_relationship,result['relationships'])
        except Exception as e:
            print(f'Error {e}')
driver.close()

Inserted entity: Dana White
Inserted entity: Ultimate Fighting Championship
Inserted entity: Power Slap
Inserted entity: Anne Stella
Inserted entity: Bishop Gorman High School
Inserted entity: Lorenzo Fertitta
Inserted entity: June White
Inserted entity: Las Vegas
Inserted entity: Ware
Inserted entity: Levant
Inserted entity: Hermon High School
Inserted entity: Quincy College
Inserted entity: UMass Boston
Inserted relationship: relationship1
Inserted relationship: relationship2
Inserted relationship: relationship3
Inserted relationship: relationship4
Inserted relationship: relationship5
Inserted relationship: relationship6
Inserted relationship: relationship7
Inserted relationship: relationship8
Inserted relationship: relationship9
Inserted relationship: relationship10
Inserted relationship: relationship11
Inserted entity: Peter Welch
Inserted entity: Whitey Bulger
Inserted entity: John Lewis
Inserted entity: Lorenzo Fertitta
Inserted entity: Frank Fertitta III
Inserted entity: Tito Or

# query through graph 

In [81]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [90]:
from langchain_groq import ChatGroq
dotenv.load_dotenv()
llm=ChatGroq(groq_api_key=os.getenv('GROQ_API_KEY'),
             model=os.getenv('GROQ_MODEL'),
             max_tokens=1000
             )

graph=Neo4jGraph(url=os.getenv('NEO4J_URI_ONLINE'),
                 username=os.getenv('NEO4J_USERNAME_ONLINE'),
                 password=os.getenv('NEO4J_PASSWORD_ONLINE')
                #  database=os.getenv('NEO4J_DATABASE')
)


In [ ]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

AttributeError: 'str' object has no attribute 'get_structured_schema'

In [98]:
# response=chain.invoke({'query':'who is elon?'})
# response

In [ ]:
# print(graph.get_schema)

Node properties:
Elon_Musk {description: STRING, id: STRING, role: STRING}
SpaceX {description: STRING, id: STRING, role: STRING}
Neuralink {description: STRING, id: STRING, role: STRING}
The_Boring_Company {description: STRING, id: STRING, role: STRING}
X {description: STRING, id: STRING, role: STRING}
Tesla_Inc {description: STRING, id: STRING, role: STRING}
X_Corp {description: STRING, id: STRING, role: STRING}
xAI {description: STRING, id: STRING, role: STRING}
OpenAI {description: STRING, id: STRING, role: STRING}
Zip2 {description: STRING, id: STRING, role: STRING}
PayPal {description: STRING, id: STRING, role: STRING}
University_of_Pennsylvania {description: STRING, id: STRING, role: STRING}
Compaq {description: STRING, id: STRING, role: STRING}
eBay {description: STRING, id: STRING, role: STRING}
Department_of_Government_Efficiency {description: STRING, id: STRING, role: STRING}
Donald_Trump {description: STRING, id: STRING, role: STRING}
Errol_Musk {description: STRING, id: ST